# Exercise 3: Durable Agent

**Goal:** Combine OpenAI agents with Temporal workflows to create a durable AI agent with automatic retries and state persistence.

**Timebox:** 15 minutes

## What You'll Learn

- How to wrap LLM calls in Temporal activities for durability
- How to persist agent state across failures using workflows
- How to correlate Temporal execution with OpenAI traces using trace IDs
- How automatic retries work for AI operations

## Prerequisites

Ensure the Temporal dev server is running before executing the exercise.

```bash
make temporal-up
```

Add your OpenAI credentials to `.env` if you have not already.

## Steps

1. **Wrap LLM calls in an activity** – Complete `call_agent_with_tools` below so LLM requests run through Temporal.
2. **Create the workflow** – Implement `DurableAgentWorkflow` to call the activity with a retry policy.
3. **Add observability** – Generate a `trace_id` and log key information for debugging.
4. **Run the workflow** – Use the helper cell to start a worker and execute the workflow.
5. **Inspect history** – Open http://localhost:8233 to observe execution state and retries.


## Setup

Install dependencies and import the Temporal + OpenAI helpers used in this exercise.


In [ ]:
%pip install --quiet temporalio openai rich

import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.common import RetryPolicy
from temporalio.worker import Worker

console = Console()

def get_weather(location: str) -> str:
    """Return mock weather data for a location."""
    weather_data = {
        "San Francisco": "sunny, 72°F",
        "New York": "cloudy, 65°F",
        "London": "rainy, 58°F",
        "Tokyo": "clear, 70°F",
    }
    weather = weather_data.get(location, "partly cloudy, 68°F")
    return f"The weather in {location} is {weather}"

@activity.defn
async def call_agent_with_tools(query: str, trace_id: str) -> str:
    """Call the LLM with tool support and return the final message."""
    # TODO: log the incoming query and trace ID
    # TODO: build the tools list that exposes the get_weather helper
    # TODO: call the OpenAI client and handle any tool invocations
    # TODO: return the agent's final response string
    raise NotImplementedError("Implement call_agent_with_tools")

@workflow.defn
class DurableAgentWorkflow:
    @workflow.run
    async def run(self, query: str, trace_id: str) -> str:
        # TODO: log the query and trace ID for observability
        # TODO: execute the call_agent_with_tools activity with a RetryPolicy
        # TODO: return the agent's response to the caller
        raise NotImplementedError("Implement DurableAgentWorkflow.run")

## Implement the Activity

Wrap the LLM call in a Temporal activity so it can retry safely.


## Implement the Workflow

Add retries and logging to orchestrate the activity invocation.


## Expected Output

```
🚀 Starting Durable Agent workflow...
   Trace ID: 550e8400-e29b-41d4-a716-446655440000

🤖 Agent Query: What's the weather in San Francisco?

🔧 Activity: Calling tool get_weather(location="San Francisco")
✓ Tool result: The weather in San Francisco is sunny, 72°F

🤖 Agent Response:
The current weather in San Francisco is sunny with a temperature of 72°F.

✅ Workflow completed successfully

View in Temporal UI: http://localhost:8233/namespaces/default/workflows/durable-agent-1
Trace ID for correlation: 550e8400-e29b-41d4-a716-446655440000
```


## Practice

Use the cell below to run the exercise script once you complete the TODOs in the Python files.

In [ ]:
# Run the durable agent exercise
async def run_exercise() -> None:
    console.print("\n[bold cyan]🚀 Exercise 3: Durable Agent[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "durable-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[DurableAgentWorkflow],
        activities=[call_agent_with_tools],
    ):
        query = "What's the weather like in San Francisco?"
        workflow_id = f"durable-agent-{trace_id}"

        console.print(f"[yellow]Query:[/yellow] {query}\n")

        result = await client.execute_workflow(
            DurableAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]🤖 Agent Response:[/bold green]\n{result}\n")
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[yellow]Trace ID for correlation:[/yellow] {trace_id}\n")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    asyncio.run(run_exercise())
else:
    import nest_asyncio
    nest_asyncio.apply()
    await run_exercise()

## Stretch Goal

- Simulate a failure inside the activity and observe automatic retries.
- Add conversation history to maintain multi-turn context.
- Implement exponential backoff retry policy for LLM calls.